In [1]:
import pandas as pd
import re
import numpy as np
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
import os
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

## Primer CSV 

In [2]:
df1 = pd.read_csv("csvs/training_data_traducido_limpio.csv")
df1.head(10) 

,nombre de empresa,descripción del trabajo,posicion_title,Descripción_length,modelo_esponse
0,google,calificaciones minimas licenciatura o experien...,especialista en ventas,2727,responsabilidades basicas responsable de expan...
1,manzana,descripcion como asc sera muy influyente en el...,consultor de soluciones de apple,828,responsabilidades centrales como asc sera muy ...
2,netflix,es un momento sorprendente para unirse a netfl...,coordinador de licencias productos de consumo,3205,core responsibilities help drive business by s...
3,robert mitad,descripcion disenadores web que buscan expandi...,disenador web,2489,responsabilidades basicas diseno de sitios web...
4,rastreador,en trackfive teniamos grandes objetivos estaba...,desarrollador web,3167,responsabilidades centrales construir y diseno...
5,designacion,designups es una agencia de diseno e interacti...,desarrollador web frontend,892,responsabilidades centrales traducir los disen...
6,equisolve inc,sobre la posicion el disenador web es responsa...,disenador de sitios web remoto,3471,responsabilidades basicas proporcione soporte ...
7,agencia de seguros de zander,descripcion del trabajo zander insurance group...,disenador web,2896,core responsibilities design compelling direct...
8,toba,tuff is a growth marketing team working with c...,disenador web,5143,core responsibilities work on various website ...
9,tecnologia de la informacion de la dinamica ge...,tipo de solicitud regular el nivel de autoriza...,sr disenador web,4023,responsabilidades basicas disena y construye g...


In [3]:
df1 = df1.drop(columns=["Descripción_length"])

In [4]:
df1

,nombre de empresa,descripción del trabajo,posicion_title,modelo_esponse
0,google,calificaciones minimas licenciatura o experien...,especialista en ventas,responsabilidades basicas responsable de expan...
1,manzana,descripcion como asc sera muy influyente en el...,consultor de soluciones de apple,responsabilidades centrales como asc sera muy ...
2,netflix,es un momento sorprendente para unirse a netfl...,coordinador de licencias productos de consumo,core responsibilities help drive business by s...
3,robert mitad,descripcion disenadores web que buscan expandi...,disenador web,responsabilidades basicas diseno de sitios web...
4,rastreador,en trackfive teniamos grandes objetivos estaba...,desarrollador web,responsabilidades centrales construir y diseno...
...,...,...,...,...
848,menards,descripcion del trabajo tiempo parcial ganar m...,pasantia de gestion,responsabilidades basicas las responsabilidade...
849,parker,responsabilidad el programa de pasantias de pa...,pasantia de recursos humanos corporativo duran...,responsabilidades basicas asistir en funciones...
850,guardar el proyecto,el proyecto borgen es una campana nacional inn...,pasantia de escritor periodista,responsabilidades basicas escriba un articulo ...
851,destinos de wyndham,pon al mundo en vacaciones en wyndham destinat...,servicio al cliente ventas entrantes remoto,responsabilidades centrales responda las llama...


## Segundo CSV

In [5]:
df2 = pd.read_csv("csvs/Curriculum_Vitae_Traducido_LIMPIO.csv")
df2.head(10)

,Categoría,Reanudar
0,ciencia de datos,habilidades lenguajes de programacion python p...
1,ciencia de datos,detalles de la educacion mayo de 2oie a mayo d...
2,ciencia de datos,areas of interest deep learning control system...
3,ciencia de datos,skills ac r ac python ac sap hana ac tableau a...
4,ciencia de datos,detalles de la educacion mca ymcaust faridabad...
5,ciencia de datos,habilidades c conceptos basicos iot python mat...
6,ciencia de datos,habilidades a c python a c tableau a c visuali...
7,ciencia de datos,education details btech rayat and bahra instit...
8,ciencia de datos,habilidades personales a c capacidad para comp...
9,ciencia de datos,experiencia de datos y analisis cuantitativos ...


## Fine-Tuning combinando con codigo hibrido.

In [7]:
model_mini = SentenceTransformer('all-MiniLM-L6-v2') 
model_spanish = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')
clasificador = pipeline("zero-shot-classification", 
                       model="Recognai/bert-base-spanish-wwm-cased-xnli")

# 2. Cargar tus datos
df_puestos = pd.read_csv('csvs/training_data_traducido_limpio.csv')  # Contiene: nombre de empresa, descripción del trabajo, posicion_title, modelo_esponse
df_cvs = pd.read_csv('csvs/Curriculum_Vitae_Traducido_LIMPIO.csv')   # Contiene: Categoría, Reanudar

# 3. Función para extraer habilidades del campo modelo_esponse
def extraer_habilidades(texto):
    # Expresión regular para encontrar patrones como "Habilidades: python, sql"
    import re
    match = re.search(r'(habilidades|skills|competencias)[:;\s]+(.+?)(?=\n|$)', texto, re.IGNORECASE)
    if match:
        return [h.strip() for h in match.group(2).split(',') if h.strip()]
    return []

# 4. Función de evaluación mejorada
def evaluar_candidato(cv_text, descripcion_puesto, habilidades_clave):
    # ---- Parallel Embeddings ----
    emb_mini_cv = model_mini.encode(cv_text)
    emb_mini_job = model_mini.encode(descripcion_puesto)
    sim_mini = util.cos_sim(emb_mini_job, emb_mini_cv).item()
    
    emb_spanish_cv = model_spanish.encode(cv_text)
    emb_spanish_job = model_spanish.encode(descripcion_puesto)
    sim_spanish = util.cos_sim(emb_spanish_job, emb_spanish_cv).item()
    
    # ---- Habilidades con Zero-Shot ----
    if habilidades_clave:  # Solo si hay habilidades definidas
        resultados_hab = clasificador(
            cv_text,
            candidate_labels=habilidades_clave,
            multi_label=True
        )
        puntaje_habilidades = sum(
            score * 2.5  # Peso ajustable
            for score in resultados_hab['scores']
            if score > 0.7
        )
        habilidades_detectadas = [
            (label, round(score, 2))
            for label, score in zip(resultados_hab['labels'], resultados_hab['scores'])
            if score > 0.7
        ]
    else:
        puntaje_habilidades = 0
        habilidades_detectadas = []
    
    # ---- Puntuación combinada ----
    puntuacion = (sim_mini * 4 + sim_spanish * 4 + puntaje_habilidades * 2) * 1.25  # Escala 0-10
    
    return {
        "puntuacion": round(puntuacion, 1),
        "detalles": {
            "all_mini_score": round(sim_mini * 10, 1),
            "spanish_model_score": round(sim_spanish * 10, 1),
            "habilidades_detectadas": habilidades_detectadas
        }
    }

# 5. Procesamiento de datos y evaluación
# Preparar df_puestos: extraer habilidades de modelo_esponse
df_puestos['habilidades_clave'] = df_puestos['modelo_esponse'].apply(extraer_habilidades)

# Ejemplo: Evaluar los primeros 5 CVs contra los primeros 10 puestos
for cv_idx, cv_row in df_cvs.head(5).iterrows():
    print(f"\n\n🔍 Evaluando CV de categoría: {cv_row['Categoría']}")
    
    for puesto_idx, puesto_row in df_puestos.head(10).iterrows():
        resultado = evaluar_candidato(
            cv_text=cv_row['Reanudar'],
            descripcion_puesto=puesto_row['descripción del trabajo'],
            habilidades_clave=puesto_row['habilidades_clave']
        )
    
        print(f"\n Empresa: {puesto_row['nombre de empresa']}")
        print(f" Puesto: {puesto_row['posicion_title']}")
        print(f" Puntuación: {resultado['puntuacion']}/10")
        if resultado['detalles']['habilidades_detectadas']:
            print(f" Habilidades detectadas: {', '.join([h[0] for h in resultado['detalles']['habilidades_detectadas']])}")

Device set to use cpu




🔍 Evaluando CV de categoría: ciencia de datos

 Empresa: google
 Puesto: especialista en ventas
 Puntuación: 4.8/10

 Empresa: manzana
 Puesto: consultor de soluciones de apple
 Puntuación: 4.2/10

 Empresa: netflix
 Puesto: coordinador de licencias productos de consumo
 Puntuación: 4.8/10

 Empresa: robert mitad
 Puesto: disenador web
 Puntuación: 5.3/10

 Empresa: rastreador
 Puesto: desarrollador web
 Puntuación: 5.7/10

 Empresa: designacion
 Puesto: desarrollador web frontend
 Puntuación: 4.9/10

 Empresa: equisolve inc
 Puesto: disenador de sitios web remoto
 Puntuación: 11.3/10
 Habilidades detectadas: requeridas 2 anos de experiencia en diseno de la web o del producto portafolio que muestra tipografia interaccion y habilidades de diseno visual comprension del diseno receptivo y sistemas de diseno modular competencia de figma y design herramientas habilidades de comunicacion fuertes requisitos educativos na nivel de experiencia 2 anos de experiencia como contribuyente individu

## guardar modelo

In [9]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import joblib

# Guardar modelos de embeddings
model_mini = SentenceTransformer('all-MiniLM-L6-v2')
model_mini.save("./modelos_skinner/all_mini_model")

model_spanish = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')
model_spanish.save("./modelos_skinner/spanish_model")

# Guardar el clasificador de habilidades (Hugging Face)
clasificador = pipeline(
    "zero-shot-classification", 
    model="Recognai/bert-base-spanish-wwm-cased-xnli"
)
clasificador.save_pretrained("./modelos_skinner/zero_shot_classifier")

# Guardar función de extracción de habilidades (opcional)
joblib.dump(extraer_habilidades, "./modelos_skinner/habilidades.pkl")

Device set to use cpu


['./modelos_skinner/habilidades.pkl']